## Laura Contreras

### Total points 8

# In  the homework
- We'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://apps.twitter.com/app/new  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [1]:
client_key = "6hIoMp2fxVL6cWbGbfUQiam4q"
client_secret = "NrKxuL9vjQyFUQfM4E6ZO4pcVCaj3jnxwTDV8SHJOALT63WdZz"

In [2]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [3]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [4]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [5]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAPbfegEAAAAA%2B9lohXdz%2BQrbZ2zzlj9toQsPFpE%3DzXYlnWZZo7u6JBsevx4ivqF4AzQiYg8lvuXHxkgcpb6MmvxZRb'}

In [6]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [7]:
max_tweets=100
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []
users =[]

In [8]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'query': 'climate change',
                'max_results': 100,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}

search_url = base_url+'2/tweets/search/recent'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [9]:
climate_tweets = response.json()


In [10]:
len(climate_tweets['data'])

100

In [11]:
tweets +=climate_tweets['data']


In [12]:

 ids = [tw['id'] for tw in climate_tweets['data']]
    

In [13]:
min_id = min(ids)

In [14]:
tw_ids = []
tw_ids+=ids

As per twitter api documentation we can use

until_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [15]:
 parameters = { 'query': 'climate change',
                'max_results': tw_limit,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
                'until_id': min_id
              }

for  i in range(max_tweets//tw_limit -1):
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'2/tweets/search/recent'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['statuses']]
    tw_ids+=ids
    print(i, len(climate_tweets['data']))
    min_id = min(ids)
    tweets +=climate_tweets['data']
    users.extend(climate_tweets['includes']['users'])
    if 'next_token' in response.json()['meta']:
					parameters['next_token'] = response.json()['meta']['next_token']

In [16]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 100


# Let's print some tweets

In [17]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [18]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'author_id': '1408248857079664645',
  'conversation_id': '1549451683238776832',
  'created_at': '2022-07-19T17:50:43.000Z',
  'entities': {'annotations': [{'end': 138,
                                'normalized_text': 'Ukr',
                                'probability': 0.8103,
                                'start': 136,
                                'type': 'Place'}],
               'hashtags': [{'end': 94, 'start': 86, 'tag': 'COVID19'}],
               'mentions': [{'end': 12,
                             'id': '189868631',
                             'start': 3,
                             'username': 'DrTedros'}]},
  'id': '1549451683238776832',
  'lang': 'en',
  'public_metrics': {'like_count': 0,
                     'quote_count': 0,
                     'reply_count': 0,
                     'retweet_count': 5},
  'referenced_tweets': [{'id': '1549451406666596354', 'type': 'retweeted'}],
  'reply_settings': 'everyone',
  'source': 'Twitter for iPhone',
  'text': 'RT 

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [19]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

RT @DrTedros: Hunger, food insecurity and malnutrition are on the rise, compounded by #COVID19, climate change, drought, and the war in Ukr…

RT @vanessa_vash: This year, the heaviest rains in at least six decades killed 459 people around the South African city of Durban and the H…

@PoliticsPollss Cost of living AND climate change

Barack Obama Has a Spot-On Analogy For Why Young People Should Care About Climate Change #bitcoin #Barack #Obama #SpotOn #Analogy #Young #People #Care #Climate #Change https://t.co/sQDCXbfN3V



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [20]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['public_metrics', 'referenced_tweets', 'id', 'created_at', 'entities', 'conversation_id', 'text', 'source', 'lang', 'reply_settings', 'author_id'])
11


In [21]:
import pandas as pd
import copy

# Generators for building DataFrames

# Q2 (2 point) Complete following   python generator *tweet_generator*. Generator yield modified tweet .
For each tweet in tweets
- Replace **author_id** attribute with **user_id**.
- Replace **referenced_tweets** attribute with **referenced_tweets_ids** and value(s) with **tweet['referenced_tweets']['id']** value(s).
- Keep other attributes in the tweet as such.

In [22]:
def tweet_generator(tweets):
    for tweet in tweets:
        try:
            modified_tweet = copy.deepcopy(tweet)
            modified_tweet['user_id'] = modified_tweet.pop('author_id')
            modified_tweet['referenced_tweets_ids'] = modified_tweet['referenced_tweets'][0]['id']
            modified_tweet['retweet_count'] = modified_tweet['public_metrics']['retweet_count']
            yield modified_tweet     
        except KeyError as e:
            continue

In [23]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'public_metrics': {'retweet_count': 5,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'referenced_tweets': [{'type': 'retweeted', 'id': '1549451406666596354'}],
  'id': '1549451683238776832',
  'created_at': '2022-07-19T17:50:43.000Z',
  'entities': {'annotations': [{'start': 136,
     'end': 138,
     'probability': 0.8103,
     'type': 'Place',
     'normalized_text': 'Ukr'}],
   'mentions': [{'start': 3,
     'end': 12,
     'username': 'DrTedros',
     'id': '189868631'}],
   'hashtags': [{'start': 86, 'end': 94, 'tag': 'COVID19'}]},
  'conversation_id': '1549451683238776832',
  'text': 'RT @DrTedros: Hunger, food insecurity and malnutrition are on the rise, compounded by #COVID19, climate change, drought, and the war in Ukr…',
  'source': 'Twitter for iPhone',
  'lang': 'en',
  'reply_settings': 'everyone',
  'user_id': '1408248857079664645',
  'referenced_tweets_ids': '1549451406666596354',
  'retweet_count': 5},
 {'public_metrics': {'retweet_count': 4,
   're

In [24]:
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [25]:
climate_feed_df

,public_metrics,referenced_tweets,id,created_at,entities,conversation_id,text,source,lang,reply_settings,user_id,referenced_tweets_ids,retweet_count,in_reply_to_user_id
0,"{'retweet_count': 5, 'reply_count': 0, 'like_c...","[{'type': 'retweeted', 'id': '1549451406666596...",1549451683238776832,2022-07-19T17:50:43.000Z,"{'annotations': [{'start': 136, 'end': 138, 'p...",1549451683238776832,"RT @DrTedros: Hunger, food insecurity and maln...",Twitter for iPhone,en,everyone,1408248857079664645,1549451406666596354,5,NaN
1,"{'retweet_count': 4, 'reply_count': 0, 'like_c...","[{'type': 'retweeted', 'id': '1549450640857890...",1549451682060410881,2022-07-19T17:50:43.000Z,"{'annotations': [{'start': 101, 'end': 113, 'p...",1549451682060410881,"RT @vanessa_vash: This year, the heaviest rain...",Twitter for Android,en,everyone,1908170252,1549450640857890821,4,NaN
2,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","[{'type': 'replied_to', 'id': '154939766819929...",1549451680378490881,2022-07-19T17:50:42.000Z,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",1549397668199292928,@PoliticsPollss Cost of living AND climate change,Twitter Web App,en,everyone,49340574,1549397668199292928,0,863849562695487489
3,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","[{'type': 'replied_to', 'id': '154943583561079...",1549451674967838723,2022-07-19T17:50:41.000Z,"{'annotations': [{'start': 12, 'end': 17, 'pro...",1549435835610791939,@SenSanders Bernie ITS ONLY AMERICA THAT IS ST...,Twitter Web App,en,everyone,223666765,1549435835610791939,0,29442313
4,"{'retweet_count': 415, 'reply_count': 0, 'like...","[{'type': 'retweeted', 'id': '1549369512558006...",1549451674464489474,2022-07-19T17:50:41.000Z,"{'mentions': [{'start': 3, 'end': 15, 'usernam...",1549451674464489474,RT @WildKenHill: Unfortunately we have a bad f...,Twitter for Android,en,everyone,242235411,1549369512558006273,415,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"{'retweet_count': 734, 'reply_count': 0, 'like...","[{'type': 'retweeted', 'id': '1549368996662755...",1549451585121665024,2022-07-19T17:50:19.000Z,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",1549451585121665024,"RT @MikeHudema: Keep in mind, the heatwaves, w...",Twitter for Android,en,everyone,1154018512626769924,1549368996662755331,734,NaN
91,"{'retweet_count': 290, 'reply_count': 0, 'like...","[{'type': 'retweeted', 'id': '1549130164558340...",1549451583003496448,2022-07-19T17:50:19.000Z,"{'mentions': [{'start': 3, 'end': 15, 'usernam...",1549451583003496448,RT @junenotmary: Everyone says climate change ...,Twitter for iPhone,en,everyone,290123942,1549130164558340098,290,NaN
92,"{'retweet_count': 3, 'reply_count': 0, 'like_c...","[{'type': 'retweeted', 'id': '1549443889463001...",1549451582999347200,2022-07-19T17:50:19.000Z,"{'mentions': [{'start': 3, 'end': 13, 'usernam...",1549451582999347200,RT @tobygrist: @SkyNews He didn't mention all ...,Twitter for iPhone,en,everyone,1267439600961757189,1549443889463001089,3,NaN
93,"{'retweet_count': 1, 'reply_count': 0, 'like_c...","[{'type': 'retweeted', 'id': '1549375854555176...",1549451582445699072,2022-07-19T17:50:19.000Z,"{'mentions': [{'start': 3, 'end': 15, 'usernam...",1549451582445699072,RT @JillDamatac: Joke threads gone viral aside...,Twitter Web App,en,everyone,717692852,1549375854555176960,1,NaN


# Q3(1 points) List datatype using climate_feed_df. Describe which attributes doesn't have correct data types. What should be correct datatype?

In [26]:
# write code here datatype listing
climate_feed_df.dtypes

public_metrics           object
referenced_tweets        object
id                       object
created_at               object
entities                 object
conversation_id          object
text                     object
source                   object
lang                     object
reply_settings           object
user_id                  object
referenced_tweets_ids    object
retweet_count             int64
in_reply_to_user_id      object
dtype: object

# Write columns with wrong datatype and mention correct data type too.

All of the id's should be either string or integer. The tweets/texts should be strings as well. 



# Q4 (1.5 point) Complete following tweet_user_generator. Generator should yield only *user* value from a tweet

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [27]:
def tweet_user_generator(tweet_user_list):
    for user in tweet_user_list:
        try:
            yield user
        except KeyError:
            continue

In [28]:
users = pd.DataFrame(tweet_user_generator(climate_tweets))

In [29]:
users

,0
0,data
1,includes
2,meta


# Q5(1 point) List datatypes of columns using user_df. Describe which attributes doesn't have correct data types. What should be correct datatype for each case?

In [30]:
# write code here for datatype listing

users.dtypes

0    object
dtype: object

# write columns with wrong datatype and mention correct data type too.

Column 0 should be string

# Q6 (1 point) Complete following retweeted_status_generator. Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary.

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [31]:
retweeted = climate_feed_df[['id', 'retweet_count']]

In [32]:
retweeted.head()

,id,retweet_count
0,1549451683238776832,5
1,1549451682060410881,4
2,1549451680378490881,0
3,1549451674967838723,0
4,1549451674464489474,415


# Q7 (1 point) Write code to find number of rows in retweeted_status_df that were not retweeted. print number of such rows.

In [33]:
# Write code here
(retweeted['retweet_count'] == 0).sum()

21